<a href="https://colab.research.google.com/github/Athulkrishna-S/CerebraLink/blob/main/Label_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
import shutil

In [3]:
df=pd.read_csv('/content/drive/MyDrive/Final_Project/datasets_csv_files/Abhijith_eye_up_down(multiples).csv',header=None)

In [ ]:
df.head()

,0
0,-12
1,-15
2,-18
3,-21
4,-24


In [ ]:
df.shape

(1806352, 1)

In [4]:
nan_count = df.isna().sum().sum()

# Print the count
print(f"Total NaN values in the DataFrame: {nan_count}")

Total NaN values in the DataFrame: 0


In [5]:
sampling_rate=10000 # 10KHz
segment_duration = 5 # 5s
samples_per_segment = segment_duration * sampling_rate

In [35]:
def segment_and_label_data(df, samples_per_segment, label_pattern):
    """
    Segments the data into chunks and labels each segment using a repeating label pattern.

    Params:
        df (pd.DataFrame): The DataFrame containing the data.
        samples_per_segment (int): The number of samples per segment.
        label_pattern (list): A list of labels to assign to segments in a repeating pattern.

    Returns:
        pd.DataFrame: A DataFrame with 'Segment' and 'Label' columns.
    """
    segments = []
    labels = []

    num_segments=df.shape[0]//samples_per_segment
    print("Number of Segments : ",num_segments)
    for i in range(num_segments):
        segment_start = i * samples_per_segment
        segment_end = segment_start + samples_per_segment


        segment = df.iloc[segment_start:segment_end].values.flatten()
        segments.append(segment)
        label_index = i % len(label_pattern)
        label = label_pattern[label_index]
        labels.append(label)

    labelled_data = pd.DataFrame({'Segment': segments, 'Label': labels})
    return labelled_data


In [28]:
label_pattern = ['rest', 'up', 'rest', 'down']
labelled_data = segment_and_label_data(df, samples_per_segment, label_pattern)
print(labelled_data)

Number of Segments :  36
Segment start :  0
Segment end :  50000
Length of segment :  50000
Segment start :  50000
Segment end :  100000
Length of segment :  50000
Segment start :  100000
Segment end :  150000
Length of segment :  50000
Segment start :  150000
Segment end :  200000
Length of segment :  50000
Segment start :  200000
Segment end :  250000
Length of segment :  50000
Segment start :  250000
Segment end :  300000
Length of segment :  50000
Segment start :  300000
Segment end :  350000
Length of segment :  50000
Segment start :  350000
Segment end :  400000
Length of segment :  50000
Segment start :  400000
Segment end :  450000
Length of segment :  50000
Segment start :  450000
Segment end :  500000
Length of segment :  50000
Segment start :  500000
Segment end :  550000
Length of segment :  50000
Segment start :  550000
Segment end :  600000
Length of segment :  50000
Segment start :  600000
Segment end :  650000
Length of segment :  50000
Segment start :  650000
Segment e

In [ ]:
labelled_data.shape

(36, 2)

1. Serialization: Saving Data (Write to File)
When you save your labelled_data DataFrame containing NumPy arrays (in the Segment column) to a CSV file:

Why serialization is needed? CSV files are text-based, so they can't directly store complex objects like NumPy arrays. Instead, we must convert (serialize) these arrays into a format that can be saved as text (e.g., JSON strings or simple lists).
2. Deserialization: Loading Data (Read from File)
When you read the file back into a DataFrame:

Why deserialization is needed? The arrays stored as text (e.g., JSON strings) in the CSV must be reconstructed (deserialized) into their original format (NumPy arrays).


If you want to avoid the complexity of converting between JSON strings and NumPy arrays, you can:

Save your data to a binary format like NumPy .npz or pickle.
Use parquet, a modern file format that can store complex data types. ***bold text***

In [29]:

# Save the DataFrame with pickle
labelled_data.to_pickle('labelled_data.pkl')


In [30]:
# Load the DataFrame with pickle
df_temp = pd.read_pickle('labelled_data.pkl')

# Calculate the length of the arrays
df_temp['length'] = df_temp['Segment'].apply(len)

print(df_temp)


                                              Segment Label  length
0   [510, 505, 500, 494, 489, 484, 479, 473, 468, ...  rest   50000
1   [248, 253, 258, 263, 268, 272, 277, 282, 288, ...    up   50000
2   [77, 80, 84, 87, 90, 93, 96, 99, 102, 104, 107...  rest   50000
3   [135, 139, 142, 146, 149, 153, 157, 161, 165, ...  down   50000
4   [796, 800, 804, 808, 813, 816, 820, 824, 828, ...  rest   50000
5   [66, 66, 66, 67, 67, 68, 69, 69, 71, 72, 73, 7...    up   50000
6   [-6960, -6964, -6969, -6973, -6978, -6982, -69...  rest   50000
7   [76, 72, 68, 64, 60, 56, 52, 48, 44, 41, 37, 3...  down   50000
8   [215, 216, 216, 217, 217, 217, 217, 216, 216, ...  rest   50000
9   [280, 281, 282, 283, 285, 286, 288, 289, 290, ...    up   50000
10  [354, 353, 353, 352, 351, 351, 350, 350, 350, ...  rest   50000
11  [-821, -822, -823, -825, -827, -829, -831, -83...  down   50000
12  [498, 495, 491, 487, 483, 479, 475, 471, 467, ...  rest   50000
13  [143, 140, 138, 136, 134, 133, 131, 131, 130

In [15]:
'''
    Split the up_down and right_left datasets into separate folders

'''

source_dir = '/content/drive/My Drive/Final_Project/datasets_csv_files'
up_down_dir = '/content/drive/My Drive/Final_Project/datasets_csv_eye_up_down'
right_left_dir = '/content/drive/My Drive/Final_Project/datasets_csv_eye_right_left'

os.makedirs(up_down_dir, exist_ok=True)
os.makedirs(right_left_dir, exist_ok=True)

for filename in os.listdir(source_dir):
  if 'eye_up_down' in filename:
    shutil.copy(os.path.join(source_dir, filename), os.path.join(up_down_dir, filename))
  elif 'eye_right_left' in filename:
    shutil.copy(os.path.join(source_dir, filename), os.path.join(right_left_dir, filename))


In [36]:
'''
    Label the up_down datasets

'''


source_dir = up_down_dir = '/content/drive/My Drive/Final_Project/datasets_csv_eye_up_down'
output_dir = '/content/drive/My Drive/Final_Project/labelled_datasets_csv_eye_up_down'

os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(source_dir):
  print('Current File : ',filename)
  full_name = os.path.join(source_dir, filename)
  df = pd.read_csv(full_name, header=None)
  print('DataFrame Shape : ',df.shape)
  label_pattern = ['rest', 'up', 'rest', 'down']
  labelled_data = segment_and_label_data(df, samples_per_segment, label_pattern)
  output_filename = os.path.splitext(filename)[0] + '_labelled.pkl'
  output_path = os.path.join(output_dir, output_filename)
  labelled_data.to_pickle(output_path)

Current File :  Alphons_eye_up_down(multiples).csv
DataFrame Shape :  (1815280, 1)
Number of Segments :  36
Current File :  Abhijith_eye_up_down(multiples).csv
DataFrame Shape :  (1806352, 1)
Number of Segments :  36
Current File :  Govind_eye_up_down(multiples).csv
DataFrame Shape :  (1806704, 1)
Number of Segments :  36
Current File :  joel_eye_up_down(multiples).csv
DataFrame Shape :  (1817664, 1)
Number of Segments :  36
Current File :  Athul_eye_up_down(multiples).csv
DataFrame Shape :  (1815632, 1)
Number of Segments :  36
Current File :  Malavika_eye_up_down(multiples).csv
DataFrame Shape :  (1859488, 1)
Number of Segments :  37


In [37]:
'''
    Label the  right_left datasets

'''

source_dir = up_down_dir = '/content/drive/My Drive/Final_Project/datasets_csv_eye_right_left'
output_dir = '/content/drive/My Drive/Final_Project/labelled_datasets_csv_eye_right_left'

os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(source_dir):
  print('Current File : ',filename)
  full_name = os.path.join(source_dir, filename)
  df = pd.read_csv(full_name, header=None)
  print('DataFrame Shape : ',df.shape)
  label_pattern = ['rest', 'right', 'rest', 'left']
  labelled_data = segment_and_label_data(df, samples_per_segment, label_pattern)
  output_filename = os.path.splitext(filename)[0] + '_labelled.pkl'
  output_path = os.path.join(output_dir, output_filename)
  labelled_data.to_pickle(output_path)

Current File :  Alphons_eye_right_left(multiples).csv
DataFrame Shape :  (1843904, 1)
Number of Segments :  36
Current File :  Anns_eye_right_left(multiples).csv
DataFrame Shape :  (1808976, 1)
Number of Segments :  36
Current File :  Abhijith_eye_right_left(multiples).csv
DataFrame Shape :  (1813664, 1)
Number of Segments :  36
Current File :  joel_eye_right_left(multiples).csv
DataFrame Shape :  (1850896, 1)
Number of Segments :  37
Current File :  Athul_eye_right_left(multiples).csv
DataFrame Shape :  (1813152, 1)
Number of Segments :  36
Current File :  Govind_eye_right_left(multiples).csv
DataFrame Shape :  (1799888, 1)
Number of Segments :  35
Current File :  Malavika_eye_right_left(multiples).csv
DataFrame Shape :  (1827920, 1)
Number of Segments :  36
